In [ ]:
pip install -r requirements.txt

In [2]:
import torch
print(torch.__version__)

2.5.0+cu118


In [3]:
import sentencepiece
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling,DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer

def summarize_text(text, max_length=100):
    # Load the pre-trained model and tokenizer
    model_name = "facebook/bart-large-cnn"
    model = BartForConditionalGeneration.from_pretrained(model_name)
    tokenizer = BartTokenizer.from_pretrained(model_name)

    # Prepare the input text
    input_ids = tokenizer.encode(text, return_tensors='pt')

    # Generate summary
    summary_ids = model.generate(input_ids, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
text_to_summarize = "Yeah, Im in high school. (pause) What are my interests? Well, my friends say I'm obsessed with celebrities. I just want to say to them, Look you little two-timing molded fruit cakes, I am NOT obsessed with celebrities! The truth is, I only in love with ONE! Theo James! I know his age, address, full name, where he lives, and where his entire family lives! I mean we're basically married. If you ever get to meet his family, you will love them! I sure will! I plan on paying them a little visit. You know, just to ask them a few questions like where's the nearest hardware store, oh and if Theo has any cameras at his house. That is the basic questions you ask your husband's parents, right? Yeah, I know this flight to England costs a lot of money, but he is worth it, anything for my hubby! I’m sure that he has gotten the hundreds of letters that I sent. He’s just too busy to write me back. Oh, I know he will be so excited to see me, well, when he regains consciousness anyway! What's that? You are calling the flight attendant to call the police? Oh, don't worry! He totally knows I am coming. I gave him a little call the other day. I guess he thought I was some obsessed teen off the street, but I am SO not obsessed! What? you think I am obsessed too? No ma'am! I am in LOVE! Anyway, here we are! I am so excited! Wish me luck!"
summary = summarize_text(text_to_summarize)
print(summary)


c:\Users\jayjo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Yeah, Im in high school. What are my interests? Well, my friends say I'm obsessed with celebrities. The truth is, I only in love with ONE! Theo James! I know his age, address, full name, where he lives, and where his entire family lives! I mean we're basically married. If you ever get to meet his family, you will love them!


In [4]:
def generate_text(genre, prompt, max_length=100):
    # Load the pre-trained model and tokenizer
    model_name = "gpt2"
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Prepare the input text
    input_text = f"{genre}: {prompt}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate text
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

    # Decode and return the generated text
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example usage
genre = "Science Fiction"
prompt = "In a distant future,"
generated_text = generate_text(genre, prompt)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Science Fiction: In a distant future, a group of scientists are trying to figure out how to make a living. They're trying to figure out how to make a living by using the power of the universe to create a new kind of life.

The Science Fiction: In a distant future, a group of scientists are trying to figure out how to make a living. They're trying to figure out how to make a living by using the power of the universe to create a new kind of life


In [5]:
def create_prompt(task, input_text):
    if task == "story":
        return f"Write a short story about {input_text}."
    elif task == "explanation":
        return f"Explain the concept of {input_text} in simple terms."
    elif task == "question":
        return f"What are the implications of {input_text}?"
    else:
        return f"Generate text for: {input_text}"

# Example usage
task = "story"
input_text = "time travel"
prompt = create_prompt(task, input_text)
generated_story = generate_text("Fiction", prompt)
print(generated_story)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Fiction: Write a short story about time travel.

The Story of Time Travel

The story of time travel is a story about time travel. It's a story about time travel. It's a story about time travel. It's a story about time travel. It's a story about time travel. It's a story about time travel. It's a story about time travel. It's a story about time travel. It's a story about time travel. It's a story


In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the T5 model and tokenizer
model_name = "t5-small"  # You can also try 't5-base' or 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def translate(text, source_lang="English", target_lang="French"):
    # Create the translation prompt for T5
    prompt = f"translate {source_lang} to {target_lang}: {text}"
    
    # Tokenize the input text
    input_tokens = tokenizer(prompt, return_tensors="pt", padding=True)
    
    # Generate translation
    output_tokens = model.generate(**input_tokens, max_length=50)
    
    # Decode and return the translation
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# Test the translation function
text = "The weather is beautiful today."
print("Translated Text:", translate(text))


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Translated Text: Le temps est beau aujourd'hui.


In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Move the model to GPU if available
if torch.cuda.is_available():
    model = model.to("cuda")

def rewrite_in_style(prompt, style):
    # Modify the input prompt to reflect the desired style
    styled_prompt = f"Rewrite the following sentence in a {style} style:\n\n{prompt}\n\n"

    # Tokenize the input
    inputs = tokenizer.encode(styled_prompt, return_tensors="pt")

    # Move the input to GPU if available
    if torch.cuda.is_available():
        inputs = inputs.to("cuda")

    # Generate text with GPT-2
    outputs = model.generate(
        inputs, 
        max_length=50,        # Increase the output length for more content
        num_return_sequences=1, 
        temperature=0.9,       # Higher temperature for more variety
        top_k=50,              # Top-k sampling
        top_p=0.95,            # Nucleus sampling
        do_sample = True, 
        pad_token_id=tokenizer.eos_token_id  # Use EOS token for padding
    )

    # Decode the generated text
    rewritten_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return rewritten_text

# Example usage with a single sentence
prompt = "The sky was blue and the sun was shining."
styles = ["Shakespearean", "casual", "formal"]

for style in styles:
    print(f"Style: {style}\n{rewrite_in_style(prompt, style)}\n")



Style: Shakespearean
Rewrite the following sentence in a Shakespearean style:

The sky was blue and the sun was shining.


So I went to sleep.


But I woke up after a night without sleep.


I saw the clouds.

Style: casual
Rewrite the following sentence in a casual style:

The sky was blue and the sun was shining.


A picture can tell a story.

A picture can change the future.

A picture can make life better.

Style: formal
Rewrite the following sentence in a formal style:

The sky was blue and the sun was shining.


In the same way that an image can be processed into form, it can be applied to any object. It can be applied to

